## Passos iniciais

#### Para o script funcionar, você precisa:

* #### Criar um projeto na plataforma da Google Cloud
* #### Ativar a API Google Analytics Reporting dentro do Google Cloud
* #### Ativar a API do Google Sheets
* #### Ativar a API do Google Drive
* #### Criar uma conta de serviço associada ao projeto que você ativou a API.
* #### Gerar chaves de API para a conta de serviço
* #### No seu painel do GA, adicione à sua conta de serviço (parece um endereço de e-mail) acesso de leitura aos dados do GA do seu site.

Links:

Criar o seu projeto no Google Cloud:
https://cloud.google.com/resource-manager/docs/creating-managing-projects

Criar e gerenciar chaves de contas de serviço
https://cloud.google.com/iam/docs/creating-managing-service-account-keys

Dimensões e métricas do GA:
https://ga-dev-tools.web.app/dimensions-metrics-explorer/

### Bibliotecas

In [ ]:
#!pip install google-auth
#!pip install --upgrade google-api-python-client
#!pip install --upgrade google-cloud-iam
#!pip install --upgrade oauth2client
#!pip install google-cloud-storage
#!pip install --upgrade google-cloud-bigquery
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
#!pip install --upgrade google-auth
#!pip install pandas-gbq

In [1]:
#Bibliotecas adicionais
import itertools as it
import pandas as pd 
import numpy as np
from tqdm import tqdm

#GoogleAnalytics
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

#BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account

#GoogleSheets
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError

### Google Sheets

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

SAMPLE_SPREADSHEET_ID = '1e9lvuMR_aog6drLWJn6VdoZoVWtl7K-vCV54fhoHfRM'
SAMPLE_RANGE_NAME = 'Seller!B2:B'
RANGE_NAME = 'Seller!C2:C'


def main():
    creds = None
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
       
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)
        
        global sheet
        sheet = service.spreadsheets()
        global resultSheets
        resultSheets = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        global resultSheelts2
        resultSheelts2 = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=RANGE_NAME).execute()
        
    except HttpError as err:
        print(err)


if __name__ == '__main__':
    main()
    
#Armazenando os nomes das Lojas em uma lista.
sellers_names = pd.DataFrame(resultSheets)
sellers_names = sellers_names['values']
sellers_names = list(sellers_names)
sellers_names

#Armazenando o id das Lojas em uma lista.
sellers_id = pd.DataFrame(resultSheelts2)
sellers_id = sellers_id['values']
sellers_id = list(sellers_id)
sellers_ids = []
for se in sellers_id:
    sellers_ids.extend(se)

### Google Analytics

In [3]:
#Credenciais
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'MY_KEY.json'

#Informações dos Sellers
sellers_ids = sellers_ids
sellers_names = sellers_names
VIEW_ID = '0'

#Variáveis que unificam as requisições de cada Seller.
lista = []
resultado_unido = []
rows_len = []

#Loop que faz a requisição para cada loja em sellers_ids
for seller in tqdm(sellers_ids):
    VIEW_ID = seller
    
    #Corpo da requisição para API.
    body = {
            'reportRequests': [
            {

                'viewId': VIEW_ID,
                'pageSize':'100000',
                'dateRanges': [{'startDate': '2021-01-01', 'endDate': 'today'}],
                'metrics': [{'expression': 'ga:newUsers'},
                             {'expression':'ga:users'},
                             {'expression':'ga:pageviewsPerSession'},
                             {'expression':'ga:transactions'},
                             {'expression':'ga:transactionsPerSession'},
                             {'expression':'ga:transactionRevenue'},
                             {'expression':'ga:buyToDetailRate'},
                             {'expression':'ga:sessionDuration'}],
                'dimensions': [{'name':'ga:date'}]
            }]
        }

    #Requisição
    def inicializacao_retorno_analytics():
      credentials = ServiceAccountCredentials.from_json_keyfile_name(
          KEY_FILE_LOCATION, SCOPES)
      analytics = build('analyticsreporting', 'v4', credentials=credentials)
      return analytics

    def get_report(analytics, pageTokenVar):
      return analytics.reports().batchGet(body=body).execute()
    

    # Formatando o resultado da requisição

    def handle_report(analytics,pagetoken,rows):  
        response = get_report(analytics, pagetoken)

        #Header, Dimentions Headers, Metric Headers 
        columnHeader = response.get("reports")[0].get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        #Pagination
        pagetoken = response.get("reports")[0].get('nextPageToken', None)

        #Rows
        rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
        rows = rows + rowsNew
        #print("len(rows): " + str(len(rows)))

       
        if pagetoken != None:
            return handle_report(analytics,pagetoken,rows)
        else:
          
            nicerows=[]
            for row in rows:
                dic={}
                dimensions = row.get('dimensions', [])
                dateRangeValues = row.get('metrics', [])

                for header, dimension in zip(dimensionHeaders, dimensions):
                    dic[header] = dimension

                for i, values in enumerate(dateRangeValues):
                    for metric, value in zip(metricHeaders, values.get('values')):
                        if ',' in value or ',' in value:
                            dic[metric.get('name')] = float(value)
                        else:
                            dic[metric.get('name')] = float(value)
                nicerows.append(dic)

            return nicerows

  
    def main():

        analytics = inicializacao_retorno_analytics()

        global dataframe
        dataframe = []

        global rows
        rows = []
        rows = handle_report(analytics,'0',rows)


    if __name__ == '__main__':
      main()
    
    # Adicionando o resultado já formatado da requisição de cada Seller em uma única array
    lista.extend(rows)  
    
    # Criando a array com o nome dos Sellers de acordo com o tamanho da array do retorno de cada requição.
    rows_len.append(len(rows))
for a,b in zip(sellers_names,rows_len):
        resultado_unido.extend(list(it.repeat(a,b)))
        
 #Montando o DataFrame
tabela_sem_nome_seller = pd.DataFrame(lista)
tabela_nomes_sellers = pd.DataFrame(resultado_unido)
tabela_pronta = tabela_sem_nome_seller.join(tabela_nomes_sellers)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.46s/it]


In [5]:
#Formatando o DataFrame
tabela_pronta.columns = ['Data','Novos_Usuarios','Usuarios','Page_Views_Por_Sessao', 'Transacoes',
                         'Taxa_de_conversao', 'Receita','Taxa_de_compra','Duracao_da_sessao','Sellers']
tabela_pronta = tabela_pronta.astype({
    'Novos_Usuarios':'int',
    'Usuarios':'int',
    'Page_Views_Por_Sessao':'int',
    'Data':'datetime64',
    'Transacoes':'int',
    'Taxa_de_conversao':'float64',
    'Receita':'float64',
    'Taxa_de_compra':'float64',
    'Duracao_da_sessao':'float64',
    'Sellers':'string'
})

tabela_pronta

,Data,Novos_Usuarios,Usuarios,Page_Views_Por_Sessao,Transacoes,Taxa_de_conversao,Receita,Taxa_de_compra,Duracao_da_sessao,Sellers
0,2022-09-01,6,6,1,0,0.0,0.0,0.0,112.0,LOJA.CURINGA.COM.BR
1,2022-09-02,2,3,1,0,0.0,0.0,0.0,68.0,LOJA.CURINGA.COM.BR
2,2022-09-03,1,1,1,0,0.0,0.0,0.0,7.0,LOJA.CURINGA.COM.BR
3,2022-09-12,1,1,1,0,0.0,0.0,0.0,8.0,LOJA.CURINGA.COM.BR
4,2022-09-13,1,1,1,0,0.0,0.0,0.0,8.0,LOJA.CURINGA.COM.BR
...,...,...,...,...,...,...,...,...,...,...
1846,2022-11-21,24,26,2,0,0.0,0.0,0.0,5747.0,LOJA.BARIGUISEMINOVOS.COM.BR
1847,2022-11-22,26,26,4,0,0.0,0.0,0.0,3288.0,LOJA.BARIGUISEMINOVOS.COM.BR
1848,2022-11-23,30,34,2,0,0.0,0.0,0.0,4744.0,LOJA.BARIGUISEMINOVOS.COM.BR
1849,2022-11-24,16,16,3,0,0.0,0.0,0.0,1945.0,LOJA.BARIGUISEMINOVOS.COM.BR


### Big Query

In [36]:
#Enviado os dados para o BigQuery
key_path = 'GBQ.json'
credentials = service_account.Credentials.from_service_account_file(
key_path, scopes = ['https://www.googleapis.com/auth/cloud-platform'])
tabela_pronta.to_gbq(credentials = credentials,
                    destination_table = 'Retail.Eventos',
                    if_exists='replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
